In [160]:
import numpy as np
from itertools import combinations
from tqdm import tqdm
import time

In [201]:
def maxvol(A, r, verbose, num_iter=3):
    from numpy.random import default_rng
    rng = default_rng()
    m, n = A.shape
    cols = np.sort(rng.choice(n, size=r, replace=False))
    cols_mask = np.zeros(n, dtype=bool)
    cols_mask[cols] = True
    cols = cols_mask
    rows = np.sort(rng.choice(m, size=r, replace=False))
    rows_mask = np.zeros(m, dtype=bool)
    rows_mask[rows] = True
    rows = rows_mask
    sub = A[rows, :][:, cols]
    if verbose:
        print('A\n', A, '\n')
        print('r =', r, '\n')
        print('rows\n', rows, '\n')
        print('cols\n', cols, '\n')
        print('initial submatrix\n', sub, '\n')
        print('initial volume =', np.abs(np.linalg.det(sub)), '\n')
    for it in range(num_iter):
        cur_vol = np.abs(np.linalg.det(A[rows, :][:, cols]))
        # cols
        sub = A[rows, :][:, cols]
        C = A[:, cols]
        CA = C @ np.linalg.inv(sub)
        if verbose:
            print('iter =', it + 1)
            print('start cols iter')
        while True:
            #print("START")
            i, j = np.unravel_index(np.argmax(np.abs(CA), axis=None), CA.shape)
            if np.isclose(np.abs(CA[i, j]), 1.0):
                print('cur vol =', np.abs(np.linalg.det(A[rows, :][:, cols])))
                break
            num = 0
            #print(i, j, '    ', CA.shape)
            #print(cols)
            cnt = 0
            k = -1    
            #print(rows)
            while cnt != j + 1:
                k += 1
                if rows[k]:
                    cnt += 1
            #print(k)
            rows[k] = False
            rows[i] = True
            #print(rows)
            ej = np.zeros(r)
            ej[j] = 1
            CA -= 1/CA[i, j] * np.outer(CA[:, j], ( CA[i, :] - ej))
            if verbose:
                print('col iter', i, j, np.sum(rows))
                print('cur vol =', np.abs(np.linalg.det(A[rows, :][:, cols])))

        #rows
        sub = A[rows, :][:, cols]
        R = A[rows, :]
        AR = np.linalg.inv(sub) @ R
        if verbose:
            print('start rows iter')
        while True:
            i, j = np.unravel_index(np.argmax(np.abs(AR), axis=None), AR.shape)
            if np.isclose(np.abs(AR[i, j]), 1.0):
                print('cur vol =', np.abs(np.linalg.det(A[rows, :][:, cols])))
                break
            cnt = 0
            k = -1
            while cnt != i + 1:
                k += 1
                if cols[k]:
                    cnt += 1
            cols[k] = False
            cols[j] = True
            ei = np.zeros(r)
            ei[i] = 1
            AR -= 1/AR[i, j] * np.outer(AR[:, j] - ei, AR[i, :])
            if verbose:
                print('row iter', i, j, np.sum(cols))
                print('cur vol =', np.abs(np.linalg.det(A[rows, :][:, cols])))
        if np.isclose(cur_vol, np.abs(np.linalg.det(A[rows, :][:, cols]))):
            print('stopping iterations')
            break
    return rows, cols

In [205]:
n = 20
m = 20
r = 3
A = np.random.normal(loc=0, scale=10, size=(m,n))
    
max_det = 0
max_cols = 0
max_rows = 0
for i in combinations(list(range(n)), r):
    for j in combinations(list(range(m)), r):
        cols = np.zeros(n, dtype=bool)
        cols[np.array(i)] = True
        rows = np.zeros(m, dtype=bool)
        rows[np.array(j)] = True
        det = np.linalg.det(A[rows, :][:, cols])
        if abs(det) > max_det:
            max_det = abs(det)
            max_cols = cols
            max_rows = rows
rows, cols = maxvol(A, r, True)
maxvol_res = abs(np.linalg.det(A[rows, :][:, cols]))
print('brute force maxvol =', max_det)
print('maxvol algoritm res =', maxvol_res, '\n')
print('abs diff =', max_det - maxvol_res)
print('rel digg =', (max_det - maxvol_res) / max_det)

A
 [[ -4.70944356   1.92767262  -9.37753993 -15.76057855  -4.32159979
    0.8614209    0.56797656  -5.38006952  -4.79704716 -12.40626739
    2.23542284  11.25365296  -8.05415636   0.3772865  -12.46370279
   -8.05171562   3.84676769   5.97345691   6.90510214  -7.72896951]
 [  2.46304555   3.97721425 -10.12086072  14.10957489  13.04525092
    2.57633758 -13.50266333 -11.18893232   2.63708287   4.41156066
   13.66837843  -6.48250345   5.1855879  -15.84822532   9.66052604
   -9.03282964   1.01401886   1.94604083  -7.03634307  -5.3032923 ]
 [  0.86380902 -14.60412535  12.63705613   1.80822551  15.59110047
   37.33895883   0.17676667   5.23188518  -4.66741208 -13.87000447
   -5.52660761  -0.68079539  -0.12471654  -8.63242173  -4.22847557
   -5.30013891  20.7907999    1.1258439   -7.94377946   3.72185241]
 [ 22.35115953   2.8378394   18.57653446  10.51227071   8.24681319
   -8.30735127  -9.41496636 -13.70697683  -6.42096426  -6.93892614
    1.95833668  -7.46349595   5.23151255   8.07211746  1

In [182]:
n = 50
m = 50
r = 2
A = np.random.normal(loc=0, scale=5, size=(m,n))
    
max_det = 0
max_cols = 0
max_rows = 0
start = time.time()
for i in combinations(list(range(n)), r):
    for j in combinations(list(range(m)), r):
        cols = np.zeros(n, dtype=bool)
        cols[np.array(i)] = True
        rows = np.zeros(m, dtype=bool)
        rows[np.array(j)] = True
        det = np.linalg.det(A[rows, :][:, cols])
        if abs(det) > max_det:
            max_det = abs(det)
            max_cols = cols
            max_rows = rows
print('brute force time =', time.time() - start)
print('brute force maxvol =', max_det)
start = time.time()
rows, cols = maxvol(A, r, False)
print('maxvol time =', time.time() - start)
maxvol_res = abs(np.linalg.det(A[rows, :][:, cols]))
print('maxvol algoritm res =', maxvol_res, '\n')
print('abs diff =', max_det - maxvol_res)
print('rel digg =', (max_det - maxvol_res) / max_det)

brute force time = 27.57050347328186
brute force maxvol = 344.85070421053325
maxvol time = 0.001001596450805664
maxvol algoritm res = 222.5412535707422 

abs diff = 122.30945063979104
rel digg = 0.3546736287513


In [183]:
n = 20
m = 20
r = 2
N = 100
maxvols = np.zeros(N)
diffs = np.zeros(N)
for it in tqdm(range(N)):
    A = np.random.normal(loc=0, scale=5, size=(m,n))
    
    max_det = 0
    max_cols = 0
    max_rows = 0
    for i in combinations(list(range(n)), r):
        for j in combinations(list(range(m)), r):
            cols = np.zeros(n, dtype=bool)
            cols[np.array(i)] = True
            rows = np.zeros(m, dtype=bool)
            rows[np.array(j)] = True
            det = np.linalg.det(A[rows, :][:, cols])
            if abs(det) > max_det:
                max_det = abs(det)
                max_cols = cols
                max_rows = rows
    maxvols[it] = max_det
    rows, cols = maxvol(A, r, False)
    maxvol_res = abs(np.linalg.det(A[rows, :][:, cols]))
    diffs[it] = max_det - maxvol_res

print('mean_diff =', diffs.mean())
print('mean_abs_diff =', diffs.mean() / maxvols.mean())

100%|██████████| 100/100 [01:05<00:00,  1.52it/s]

mean_diff = 86.65986375178271
mean_abs_diff = 0.33106154458584747


In [184]:
n = 10
m = 10
r = 5
N = 100
maxvols = np.zeros(N)
diffs = np.zeros(N)
for it in tqdm(range(N)):
    A = np.random.normal(loc=0, scale=5, size=(m,n))
    
    max_det = 0
    max_cols = 0
    max_rows = 0
    for i in combinations(list(range(n)), r):
        for j in combinations(list(range(m)), r):
            cols = np.zeros(n, dtype=bool)
            cols[np.array(i)] = True
            rows = np.zeros(m, dtype=bool)
            rows[np.array(j)] = True
            det = np.linalg.det(A[rows, :][:, cols])
            if abs(det) > max_det:
                max_det = abs(det)
                max_cols = cols
                max_rows = rows
    maxvols[it] = max_det
    rows, cols = maxvol(A, r, False)
    maxvol_res = abs(np.linalg.det(A[rows, :][:, cols]))
    diffs[it] = max_det - maxvol_res

print('mean_diff =', diffs.mean())
print('mean_abs_diff =', diffs.mean() / maxvols.mean())

100%|██████████| 100/100 [02:04<00:00,  1.25s/it]

mean_diff = 107401.61729067065
mean_abs_diff = 0.2917069700201795
